**Finn McEvoy** - Practical Data Science for Economists 2024

<a href="https://colab.research.google.com/drive/1RXWtKSScwXJL3QZrk_WHzkPa6RXaDHN8?usp=sharing" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transforming Data

The bread and butter of a data workflow is cleaning and preparation, taking raw datasets and transforming them into a useful form.

Today we're going to produce a chart using two series from the Economics Observatory API that we'll transform with `Pandas` and present with `Vega-lite`. We will produce a chart showing Indian Exports and Imports as a percentage of GDP using data sourced from the API.

</br></br></br></br>

## Introducing Tools: `Requests`

The `Requests` module allows us to fetch resources from the internet, whether these are `CSVs`, `JSONs`, images, `HTML` or anything else. This is particuarly important for requesting data from APIs.

</br>

It is simple to use. Usually allow we need to do is:

1. Make a request with `requests.get` and our target URL. For example we can request something from our GitHub repos:

    `req = requests.get("https://raw.githubusercontent.com/mclass-user/mclass-user.github.io/main/s2_chart1.json")`
</br>
</br>
2. Access the fetched data. Using `req.json()` for JSON data or `req.text` for most other data. For example, we can see the returned JSON for the chart we just fetched:
    </br>
    </br>
    `data = req.json()`

First, let's import the modules we'll be using.

In [5]:
import requests
import pandas as pd

In [3]:
response = requests.get("https://raw.githubusercontent.com/mclass-user/mclass-user.github.io/main/s2_chart1.json")

# Let's try and view the data
print(response.json())


{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'title': {'text': 'Human Development Index ', 'subtitle': ["P21 Countries' HDI, most recent year", 'Source: UN']}, 'description': 'A simple bar chart with embedded data.', 'data': {'values': [{'Country': 'Bangladesh', 'HDI': 0.661}, {'Country': 'Brazil', 'HDI': 0.754}, {'Country': 'China', 'HDI': 0.768}, {'Country': 'DR Congo', 'HDI': 0.479}, {'Country': 'Egypt', 'HDI': 0.731}, {'Country': 'Ethiopia', 'HDI': 0.498}, {'Country': 'Germany', 'HDI': 0.942}, {'Country': 'India', 'HDI': 0.633}, {'Country': 'Indonesia', 'HDI': 0.705}, {'Country': 'Iran', 'HDI': 0.774}, {'Country': 'Japan', 'HDI': 0.925}, {'Country': 'Mexico', 'HDI': 0.758}, {'Country': 'Nigeria', 'HDI': 0.535}, {'Country': 'Pakistan', 'HDI': 0.544}, {'Country': 'Philippines', 'HDI': 0.699}, {'Country': 'Russia', 'HDI': 0.829}, {'Country': 'Thailand', 'HDI': 0.8}, {'Country': 'Turkey', 'HDI': 0.838}, {'Country': 'United Kingdom', 'HDI': 0.929}, {'Country': 'United 

So we've managed to bring JSON data from online into our Python Notebook using the `requests` module.

</br></br>
</br></br>
</br>
</br>
</br>

# Transforming Data: Indian Exports and Imports

We want to produce a chart showing Indian exports and Imports as a percentage of GDP. To do this, we can use three series from the [Economics Observatory API](https://www.economicsobservatory.com/data-hub):
1. Indian Exports
2. Indian Imports
3. Indian GDP

</br>
</br>

We can find the API urls for these from the [Data-Hub](https://www.economicsobservatory.com/data-hub)

<img
style="max-height: 350px;
    width: auto;" src="https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/misc/Masterclass/section%205/images/data-hub.png">
</img>

</br>
</br>

We find:

1. **Exports**: https://api.economicsobservatory.com/ind/expo
2. **Imports**: https://api.economicsobservatory.com/ind/impo
3. **GDP**: https://api.economicsobservatory.com/ind/gdpa

</br>
</br>


Let's try requesting the first of these, `Exports` and see how it's formatted.

In [4]:
exports_req = requests.get("https://api.economicsobservatory.com/ind/expo")
exports_req.json()

{'author': 'Economics Observatory',
 'source': 'CSO',
 'url': 'https://eco-temp-cache.s3.eu-west-2.amazonaws.com/ind/expo.json',
 'data': [{'date': '2011-04-01', 'value': 493074.88},
  {'date': '2011-07-01', 'value': 503937.31},
  {'date': '2011-10-01', 'value': 557993.27},
  {'date': '2012-01-01', 'value': 588925.54},
  {'date': '2012-04-01', 'value': 555655.03},
  {'date': '2012-07-01', 'value': 694342.2},
  {'date': '2012-10-01', 'value': 566043.39},
  {'date': '2013-01-01', 'value': 623666.38},
  {'date': '2013-04-01', 'value': 616226.48},
  {'date': '2013-07-01', 'value': 747412.79},
  {'date': '2013-10-01', 'value': 726997.88},
  {'date': '2014-01-01', 'value': 766144.14},
  {'date': '2014-04-01', 'value': 706895.73},
  {'date': '2014-07-01', 'value': 727107.86},
  {'date': '2014-10-01', 'value': 732620.04},
  {'date': '2015-01-01', 'value': 697012.65},
  {'date': '2015-04-01', 'value': 671293.85},
  {'date': '2015-07-01', 'value': 687100.61},
  {'date': '2015-10-01', 'value': 67

</br>
</br>

This response contains some metadata, then the actual data is nested in the `data` key. (Recall we looked at accessing nested dictionaries in the PythonBasics workbook.)

Since we just care about the data itself, let's select just what's inside this `data` key and then make a dataframe out of it.

In [7]:
exports_response = exports_req.json()
exports_data = exports_response['data']
exports_df = pd.DataFrame(exports_data)

exports_df.head(5)

,date,value
0,2011-04-01,493074.88
1,2011-07-01,503937.31
2,2011-10-01,557993.27
3,2012-01-01,588925.54
4,2012-04-01,555655.03


We'll rename the `value` column to `export` so we can keep track of what this data is when we merge it.

In [8]:
exports_df = exports_df.rename(columns={"value": "exports"})
exports_df.head(5)

,date,exports
0,2011-04-01,493074.88
1,2011-07-01,503937.31
2,2011-10-01,557993.27
3,2012-01-01,588925.54
4,2012-04-01,555655.03


</br>
</br>
</br>

We now have a Pandas `DataFrame` containing the exports data. Let's do the same for imports and GDP.

In [9]:
imports_url = "https://api.economicsobservatory.com/ind/impo"
imports_req = requests.get(imports_url) # This is the request
imports_response = imports_req.json() # This is the response in JSON format
imports_data = imports_response['data'] # We only want the data part of the response, not the metadata
imports_df = pd.DataFrame(imports_data) # We convert the data to a DataFrame
imports_df = imports_df.rename(columns={"value": "imports"}) # We rename the value column to imports

gdp_url = "https://api.economicsobservatory.com/ind/gdpa"
gdp_req = requests.get(gdp_url)
gdp_response = gdp_req.json()
gdp_data = gdp_response['data']
gdp_df = pd.DataFrame(gdp_data)
gdp_df = gdp_df.rename(columns={"value": "gdp"})

In [11]:
imports_df.head(2)

,date,imports
0,2011-04-01,626234.56
1,2011-07-01,636896.39


In [12]:
gdp_df.head(2)

,date,gdp
0,2011-01-01,2.043496e+06
1,2011-04-01,2.029469e+06


</br>
</br>

Now that we have all three series downloaded, we can start merging and transforming the data.

Let's merge our `gdp_df` with the `exports_df` and `imports_df` so we can express them as a ratio.

In [13]:
exports_ratio_df = exports_df.merge(gdp_df, on="date")
exports_ratio_df.tail(5)

,date,exports,gdp
40,2021-04-01,1128349.43,5.627315e+06
41,2021-07-01,1225739.20,6.268323e+06
42,2021-10-01,1312634.89,6.523591e+06
43,2022-01-01,1397161.93,6.499562e+06
44,2022-04-01,1463323.34,6.468804e+06


</br>
</br>

Let's now divide the `exports` column by the `gdp` column to get the Ratio and keep only the result and the `date`.

In [14]:
exports_ratio_df['Exports Ratio'] = exports_ratio_df['exports'] / exports_ratio_df['gdp'] # Divide exports by GDP to get the ratio
exports_ratio_df = exports_ratio_df[['date', 'Exports Ratio']] # Select only the date and Exports Ratio columns
exports_ratio_df.head(5)

,date,Exports Ratio
0,2011-04-01,0.242958
1,2011-07-01,0.224486
2,2011-10-01,0.230718
3,2012-01-01,0.254593
4,2012-04-01,0.235751


</br>
</br>
</br>

We'll also do the same to the `imports_df`:

In [15]:
imports_ratio_df = imports_df.merge(gdp_df, on="date") # Merge imports and GDP DataFrames on the date column
imports_ratio_df['Imports Ratio'] = imports_ratio_df['imports'] / imports_ratio_df['gdp'] # Divide imports by GDP to get the ratio
imports_ratio_df = imports_ratio_df[['date', 'Imports Ratio']] # Select only the date and Imports Ratio columns
imports_ratio_df.head(5)

,date,Imports Ratio
0,2011-04-01,0.308571
1,2011-07-01,0.283714
2,2011-10-01,0.297630
3,2012-01-01,0.316704
4,2012-04-01,0.303726


<br/><br/>

We now have all the data we need in two dataframes: `exports_ratio_df` and `imports_ratio_df`.

To use these in a graph, we'll need them in one dataframe.

In [16]:
trade_df = pd.merge(exports_ratio_df, imports_ratio_df, on="date") # Merge the exports and imports DataFrames on the date column
trade_df.head(5)

,date,Exports Ratio,Imports Ratio
0,2011-04-01,0.242958,0.308571
1,2011-07-01,0.224486,0.283714
2,2011-10-01,0.230718,0.297630
3,2012-01-01,0.254593,0.316704
4,2012-04-01,0.235751,0.303726


</br></br>

It's best practice for `Vega-lite` to keep data in a TIDY format where there are columns that specify the series and a column for the data. To tranform our data, we'll use the pandad function `melt`.

In [17]:
trade_df = trade_df.melt(id_vars="date", var_name="series", value_name="value") # Melt the DataFrame so that the exports and imports ratios are in the same column,
                                                                                # Specifying we want columns of date and series
trade_df

,date,series,value
0,2011-04-01,Exports Ratio,0.242958
1,2011-07-01,Exports Ratio,0.224486
2,2011-10-01,Exports Ratio,0.230718
3,2012-01-01,Exports Ratio,0.254593
4,2012-04-01,Exports Ratio,0.235751
...,...,...,...
85,2021-04-01,Imports Ratio,0.205880
86,2021-07-01,Imports Ratio,0.216721
87,2021-10-01,Imports Ratio,0.237311
88,2022-01-01,Imports Ratio,0.244439


</br>
</br>

We can now export this data to use in `Vega-lite`.

In [18]:
trade_df.to_csv("indian_trade_data.csv", index=False)

</br></br>

Finally, to get the CSV onto GitHub from Colab. We just have to go to the Files tab in the sidebar, and find our file:

<img
style="max-height: 350px;
    width: auto;" src="https://raw.githubusercontent.com/jhellingsdata/RADataHub/main/misc/Masterclass/section%205/images/colab_download.png">
</img>

Hit download, then you can add this to your VSCode and upload to GitHub.